In [1]:
from preprocessor import *

In [2]:
df=pd.read_csv('HR.csv')

In [3]:
pd.set_option('display.max_columns',None)

In [4]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [5]:
df.drop(['EmployeeNumber','Over18','StandardHours','EmployeeCount'],axis=1,inplace = True)

In [6]:
df.drop('JobLevel',axis=1,inplace=True)

In [7]:
x=df.drop('Attrition',axis=1)
y=df['Attrition'].map({'No':0,'Yes':1})

In [8]:
with open('preprocessor.pkl','rb') as f:
    preprocessor=pickle.load(f)

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=20)

In [20]:
processed_x_train=preprocessor.fit_transform(x_train)

In [22]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights=compute_sample_weight(class_weight={0:1,1:5},y=y_train)

In [24]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(processed_x_train,y_train,sample_weight=sample_weights)

DecisionTreeClassifier()

In [26]:
processed_x_test=preprocessor.fit_transform(x_test)

In [28]:
predict=dt.predict(processed_x_test)

In [30]:
from sklearn.metrics import accuracy_score,f1_score,classification_report

In [32]:
print(accuracy_score(y_test,predict))
print(classification_report(y_test,predict))
print(f1_score(y_test,predict))

0.7755102040816326
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       243
           1       0.33      0.29      0.31        51

    accuracy                           0.78       294
   macro avg       0.59      0.59      0.59       294
weighted avg       0.76      0.78      0.77       294

0.3125


### Hyperparameter for Decision Tree

In [36]:
from sklearn.model_selection import GridSearchCV

In [38]:
params={
    'criterion':('gini','entropy'),
    'splitter':('random','best'),
    'max_depth':(list(range(1,11))),
    'min_samples_split':[2,3,4,5,6,7,8,9,10],
    'min_samples_leaf':list(range(1,20))
}
tree_clf=DecisionTreeClassifier()
tree_cv=GridSearchCV(tree_clf,params,scoring='f1',n_jobs=-1,verbose=2,cv=5)

tree_cv.fit(processed_x_train,y_train,sample_weight=sample_weights)
print(tree_cv.best_params_)

Fitting 5 folds for each of 6840 candidates, totalling 34200 fits
{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 10, 'min_samples_split': 9, 'splitter': 'random'}


In [40]:
dt_hy=DecisionTreeClassifier(criterion= 'entropy', max_depth= 9, min_samples_leaf= 10, min_samples_split= 9, splitter= 'random')

In [42]:
dt_hy.fit(processed_x_train,y_train,sample_weight=sample_weights)

DecisionTreeClassifier(criterion='entropy', max_depth=9, min_samples_leaf=10,
                       min_samples_split=9, splitter='random')

In [44]:
predict_hy=dt_hy.predict(processed_x_test)

In [48]:
print(classification_report(y_test,predict_hy))

              precision    recall  f1-score   support

           0       0.91      0.79      0.84       243
           1       0.38      0.61      0.47        51

    accuracy                           0.76       294
   macro avg       0.64      0.70      0.66       294
weighted avg       0.81      0.76      0.78       294



In [50]:
print(f1_score(y_test,predict_hy))

0.46616541353383456


# RANDOM FOREST

In [67]:
from sklearn.ensemble import RandomForestClassifier

In [69]:
rf_clf=RandomForestClassifier()
rf_clf.fit(processed_x_train,y_train,sample_weight=sample_weights)

RandomForestClassifier()

In [71]:
predict3=rf_clf.predict(processed_x_test)

In [73]:
print(classification_report(y_test,predict))
print(f1_score(y_test,predict))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88       243
           1       0.40      0.37      0.39        51

    accuracy                           0.80       294
   macro avg       0.64      0.63      0.63       294
weighted avg       0.79      0.80      0.79       294

0.3877551020408163


## HYPERPARAMETERS FOR RF

In [102]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
max_features=['sqrt','log2']
max_depth=[int(x) for x in np.linspace(11,110,num=11)]
min_samples_split=[2,5,10]
min_samples_leaf=[1,2,4,5]
bootstrap=[True,False]

random_grid={'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,'min_samples_split':min_samples_split,
            'min_samples_leaf':min_samples_leaf,'bootstrap':bootstrap}
rf_clf1=RandomForestClassifier(random_state=42)
rf_cv=RandomizedSearchCV(estimator=rf_clf1,scoring='f1',param_distributions=random_grid,n_iter=200,
                        cv=5,verbose=2,random_state=42,n_jobs=-1)
rf_cv.fit(processed_x_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [11, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4, 5],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, scoring='f1', verbose=2)

In [104]:
rf_cv.best_params_

{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': False}

In [106]:
rf=RandomForestClassifier(n_estimators= 800,
 min_samples_split= 5,
 min_samples_leaf= 1,
 max_features= 'sqrt',
 max_depth= 50,
 bootstrap= False)

In [108]:
rf.fit(processed_x_train,y_train)

RandomForestClassifier(bootstrap=False, max_depth=50, min_samples_split=5,
                       n_estimators=800)

In [110]:
predict3=rf.predict(processed_x_test)

In [112]:
print(classification_report(y_test,predict3))
print(f1_score(y_test,predict3))

              precision    recall  f1-score   support

           0       0.85      0.99      0.92       243
           1       0.77      0.20      0.31        51

    accuracy                           0.85       294
   macro avg       0.81      0.59      0.61       294
weighted avg       0.84      0.85      0.81       294

0.3125


In [114]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score
import numpy as np

rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [200, 400, 600, 800, 1000],
    'max_depth': [8, 12, 16, 20, None],
    'max_features': ['sqrt', 'log2', 0.5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 3, 5],
    'bootstrap': [True],
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

scorer = make_scorer(f1_score, pos_label=1)

rf_cv = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=80,
    scoring=scorer,
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)

rf_cv.fit(processed_x_train, y_train)
print(rf_cv.best_params_)


Fitting 5 folds for each of 80 candidates, totalling 400 fits
{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': 12, 'class_weight': 'balanced', 'bootstrap': True}


In [116]:
rf2=RandomForestClassifier(n_estimators= 1000,min_samples_split= 2,min_samples_leaf= 5,max_features= 'log2',
                          max_depth= 12,bootstrap= True,class_weight= 'balanced')

In [118]:
rf2.fit(processed_x_train,y_train)

RandomForestClassifier(class_weight='balanced', max_depth=12,
                       max_features='log2', min_samples_leaf=5,
                       n_estimators=1000)

In [126]:
predict4=rf2.predict(processed_x_test)

In [128]:
print(classification_report(y_test,predict4))
print(f1_score(y_test,predict4))

              precision    recall  f1-score   support

           0       0.90      0.95      0.92       243
           1       0.68      0.49      0.57        51

    accuracy                           0.87       294
   macro avg       0.79      0.72      0.75       294
weighted avg       0.86      0.87      0.86       294

0.5681818181818182


# GRADIENT BOOSTING

In [131]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
param_grid={
    'learning_rate': [0.005, 0.01, 0.02, 0.03, 0.05],
    'n_estimators': [200, 400, 600, 900],
    'max_depth': [3, 4, 5],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3, 5],
    'max_features': [None, 'sqrt', 0.5]
}

gb=GradientBoostingClassifier()
gbc=GridSearchCV(estimator=gb,scoring='f1',param_grid=param_grid,verbose=2,refit=True,cv=4,n_jobs=-1)
gbc.fit(processed_x_train,y_train,sample_weight=sample_weights)
print(gbc.best_params_)

Fitting 4 folds for each of 8640 candidates, totalling 34560 fits


In [135]:
gbc2=GradientBoostingClassifier(learning_rate=0.07,max_depth=4,n_estimators=50)

In [137]:
gbc2.fit(processed_x_train,y_train,sample_weight=sample_weights)

GradientBoostingClassifier(learning_rate=0.07, max_depth=4, n_estimators=50)

In [139]:
predict4=gbc2.predict(processed_x_test)

In [141]:
print(classification_report(y_test,predict4))
print(f1_score(y_test,predict4))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       243
           1       0.55      0.61      0.58        51

    accuracy                           0.85       294
   macro avg       0.73      0.75      0.74       294
weighted avg       0.85      0.85      0.85       294

0.5794392523364486


In [143]:
gbc=GradientBoostingClassifier()
gbc.fit(processed_x_train,y_train,sample_weight=sample_weights)
predict5=gbc.predict(processed_x_test)
print(classification_report(y_test,predict5))
print(f1_score(y_test,predict5))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       243
           1       0.57      0.63      0.60        51

    accuracy                           0.85       294
   macro avg       0.75      0.76      0.75       294
weighted avg       0.86      0.85      0.86       294

0.5981308411214953


In [35]:
 pip install xgboost

In [43]:
from xgboost import XGBClassifier

In [45]:
xgb=XGBClassifier()

In [49]:
xgb.fit(processed_x_train,y_train,sample_weight=sample_weights)
predict_xgb=xgb.predict(processed_x_test)
print(classification_report(y_test,predict_xgb))
print(f1_score(y_test,predict_xgb))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93       243
           1       0.74      0.51      0.60        51

    accuracy                           0.88       294
   macro avg       0.82      0.74      0.77       294
weighted avg       0.88      0.88      0.88       294

0.6046511627906976


In [53]:
from sklearn.model_selection import RandomizedSearchCV

In [57]:
param_grid={
    'n_estimators':[500,600,800,1000,1500,2000],
    'max_depth':[3,4,6,7,8,9],
    'learning_rate':[0.01,0.02,0.03],
    'reg_alpha':[0,0.5,0.1,0.2,0.3,0.4],
    'reg_lambda':[0.5,1,2,3,4]
}
xgb=XGBClassifier(random_state=42)
rcv=RandomizedSearchCV(estimator=xgb,scoring='f1',param_distributions=param_grid,n_iter=100,cv=3,verbose=2,n_jobs=-1)
rcv.fit(processed_x_train,y_train,sample_weight=sample_weights)
print(rcv.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'reg_lambda': 1, 'reg_alpha': 0.5, 'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.03}


In [59]:
xbg2=XGBClassifier(reg_lambda=1,reg_alpha=0.5,n_estimators=1000,max_depth=4,learning_rate=0.03)

In [63]:
xbg2.fit(processed_x_train,y_train,sample_weight=sample_weights)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=None, ...)

In [65]:
predict7=xbg2.predict(processed_x_test)

In [69]:
print(classification_report(y_test,predict7))
print(f1_score(y_test,predict7))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92       243
           1       0.61      0.65      0.63        51

    accuracy                           0.87       294
   macro avg       0.77      0.78      0.77       294
weighted avg       0.87      0.87      0.87       294

0.6285714285714286


In [75]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, loguniform, uniform
import numpy as np

# compute scale_pos_weight if data imbalanced
n_pos = (y_train==1).sum()
n_neg = (y_train==0).sum()
scale_pos_weight = float(n_neg / max(1, n_pos))

xgb = XGBClassifier(
    random_state=42,
    
       # use 'hist' if no GPU
    eval_metric='logloss'
)

param_dist = {
    'n_estimators': randint(100, 801),     # 100-800
    'max_depth': randint(3, 9),            # 3-8
    'learning_rate': loguniform(0.01, 0.2),# sample LR on log scale
    'reg_alpha': uniform(0.0, 1.0),        # 0-1
    'reg_lambda': uniform(0.5, 3.0),       # 0.5-3
    'subsample': uniform(0.6, 0.4),        # 0.6-1.0
    'colsample_bytree': uniform(0.6, 0.4)
}

rcv = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=40,           # evaluate only 40 combos (feel free to raise to 60)
    cv=3,
    scoring='f1',
    verbose=2,
    n_jobs=1,            # avoid colab thrash
    random_state=42
)

# pass sample_weight if you used before; also you can keep scale_pos_weight inside params
rcv.fit(processed_x_train, y_train, sample_weight=sample_weights)
print("best params:", rcv.best_params_)


Fitting 3 folds for each of 40 candidates, totalling 120 fits
[CV] END colsample_bytree=0.749816047538945, learning_rate=0.17254716573280354, max_depth=5, n_estimators=171, reg_alpha=0.5986584841970366, reg_lambda=0.9680559213273096, subsample=0.662397808134481; total time=   0.0s
[CV] END colsample_bytree=0.749816047538945, learning_rate=0.17254716573280354, max_depth=5, n_estimators=171, reg_alpha=0.5986584841970366, reg_lambda=0.9680559213273096, subsample=0.662397808134481; total time=   0.1s
[CV] END colsample_bytree=0.749816047538945, learning_rate=0.17254716573280354, max_depth=5, n_estimators=171, reg_alpha=0.5986584841970366, reg_lambda=0.9680559213273096, subsample=0.662397808134481; total time=   0.0s
[CV] END colsample_bytree=0.6232334448672797, learning_rate=0.13394334706750485, max_depth=6, n_estimators=763, reg_alpha=0.6508884729488529, reg_lambda=0.6692347370813008, subsample=0.8887995089067299; total time=   0.2s
[CV] END colsample_bytree=0.6232334448672797, learning_r

In [77]:
xgb3=XGBClassifier(colsample_bytree = 0.632341330533086, learning_rate= 0.030264362859954555, max_depth= 3, n_estimators= 393, reg_alpha= 0.3677831327192532, reg_lambda=2.3969174917807385, subsample= 0.8534118843043579)

In [79]:
xgb3.fit(processed_x_train,y_train,sample_weight=sample_weights)
predict8=xgb3.predict(processed_x_test)
print(classification_report(y_test,predict8))
print(f1_score(y_test,predict8))

              precision    recall  f1-score   support

           0       0.94      0.85      0.89       243
           1       0.51      0.73      0.60        51

    accuracy                           0.83       294
   macro avg       0.72      0.79      0.74       294
weighted avg       0.86      0.83      0.84       294

0.5967741935483871
